# Prepare data for bert

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

C:\Users\mirei\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\mirei\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\mirei\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)
C:\Users\mirei\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
# huntington's
nodes_hd = pd.read_csv('final data-kg/graph_nodes_v2024-05-27.csv') #04-02
edges_hd = pd.read_csv("final data-kg/graph_edges_v2024-05-27.csv") #04-02
pred_pairs = pd.read_csv("final data-kg/pred_pairs.csv")
pred_pairs_drugs = pd.read_csv("final data-kg/pairs_pred_drugs.csv")

In [8]:
# alzheimer's
nodes_ad = pd.read_csv('final data-kg/alz_graph_nodes_v2024-05-25.csv')#"ALZHEIMER/monarch_nodes_v2024-05-25.csv")
edges_ad = pd.read_csv('final data-kg/alz_graph_edges_v2024-05-25.csv')#"ALZHEIMER/monarch_edges_v2024-05-25.csv")
ad_dist_edges = pd.read_csv('final data-kg/ad_dist_edges.csv')
# nodes_ad.head()

C:\Users\mirei\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3417: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
len(ad_dist_edges)

110814

In [4]:
def triplet_file(nodes, edges, prop=True):
    
    # get relevant info on entities/ prepare for merging
    subject = nodes.copy()
    object_ = nodes.copy()
    try:
        subject = subject.drop(columns=['semantic_groups'])
        subject = subject.rename(columns={"id": "subject_id", "preflabel": "subject_pl", "synonyms": "subject_syn", 
                                          "name": "subject_nm", "description": "subject_dsc"})

        object_ = object_.drop(columns=['semantic_groups'])
        object_ = object_.rename(columns={"id": "object_id", "preflabel": "object_pl", "synonyms": "object_syn", 
                                          "name": "object_nm", "description": "object_dsc"})
    except:
        pass
    
    # combine entities with edges
    merged1 = pd.merge(subject, edges, on='subject_id', how='right')
    merged2 = pd.merge(object_, merged1, on='object_id', how='right')
    print('merging worked:', len(merged1) == len(merged2))
    
    # fill nans with identifiers
    merged2['object_pl'] = merged2['object_pl'].fillna(merged2['object_id'])
    merged2['subject_pl'] = merged2['subject_pl'].fillna(merged2['subject_id'])
    
    if prop:
        merged2['property_label'] = merged2['property_label'].fillna(merged2['property_id'])

        # clean property label
        merged2['property_label'] = merged2['property_label'].str.replace('biolink:', '')
        merged2['property_label'] = merged2['property_label'].str.replace('_', ' ')
        
    else:
        # keep only the 3 columns
        merged2['property_label'] = 'interacts with'
        
    # keep only the 3 columns
    triplets = merged2[['subject_pl', 'property_label', 'object_pl']]
    
    return triplets

In [9]:
triplets_ad = triplet_file(nodes_ad, edges_ad)
# saving
train, val = train_test_split(triplets_ad, test_size=0.2, random_state=42, shuffle=True)
train.to_csv('data-avui/train.csv', sep='\t', index=False)
val.to_csv('data-avui/val.csv', sep='\t', index=False)
print(len(train) + len(val) == len(edges_ad))

merging worked: True
True


In [11]:
triplets_ad_dist = triplet_file(nodes_ad, ad_dist_edges)
train_d, val_d = train_test_split(triplets_ad_dist, test_size=0.2, random_state=42, shuffle=True)
train_d.to_csv('data-avui/train_ad.csv', sep='\t', index=False)
val_d.to_csv('data-avui/val_ad.csv', sep='\t', index=False)
print(len(train_d) + len(val_d) == len(ad_dist_edges))

merging worked: True
True


In [5]:
testing = triplet_file(nodes_hd, pred_pairs, prop=False)
testing.to_csv('data/test.csv', index=None) 
# testing.head()

merging worked: True


In [11]:
testing_d = triplet_file(nodes_ad, pred_pairs_drugs, prop=False)
testing_d.to_csv('data/test_drugs.csv', index=None) 
# testing_d.head()

merging worked: True


---------

# split to load to neo4j

In [20]:
print(len(edges_hd)/500000)
one = edges_hd[:500000] #500,000
two  = edges_hd[500000:1000000]
three = edges_hd[1000000:]
one.to_csv('data-neo4j/edges_hd_1.csv', index=False)
two.to_csv('data-neo4j/edges_hd_2.csv', index=False)
three.to_csv('data-neo4j/edges_hd_3.csv', index=False)

2.712896
